In [23]:
import requests
import pandas as pd
import time

In [24]:
APP_ID ="e68ebd66"
APP_KEY="eb1415d8ff1df57c810e66c044d119dd"

In [25]:
roles = [
    "Software Engineer", "Data Analyst", "Business Analyst", "Backend Developer",
    "Software Developer", "Data Engineer", "Data Scientist", "Cyber Security Engineer",
    "DFT Engineer", "Validation Engineer", "SoC Design Verification Engineer",
    "Post Silicon Validation", "Design Verification Engineer"
]

pages_per_role = 20 

all_jobs = []

for role in roles:
    print(f"\nFetching jobs for: {role}")
    
    for page in range(1, pages_per_role + 1):
        print(f"Page {page}")
        
        url = f"https://api.adzuna.com/v1/api/jobs/us/search/{page}"
        params = {
            "app_id": APP_ID,
            "app_key": APP_KEY,
            "what": role,
            "results_per_page": 50,
            "content-type": "application/json"
        }

        r = requests.get(url, params=params)

        if r.status_code != 200:
            print(f"Failed at page {page} for role '{role}': Status {r.status_code}")
            print(r.text)
            break

        jobs = r.json().get("results", [])

        for job in jobs:
            all_jobs.append({
                "Role": role,
                "Title": job.get("title"),
                "Company": job.get("company", {}).get("display_name"),
                "Location": job.get("location", {}).get("display_name"),
                "Salary": f"${job.get('salary_min')} - ${job.get('salary_max')}" if job.get("salary_is_predicted") == "1" else "Not listed",
                "Description": job.get("description", "")[:150] + "..."
            })

        time.sleep(1)


Fetching jobs for: Software Engineer
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20

Fetching jobs for: Data Analyst
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20

Fetching jobs for: Business Analyst
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20

Fetching jobs for: Backend Developer
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20

Fetching jobs for: Software Developer
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20

Fetching jobs for: Data Engineer
Page 1
Page 2
Page 3
Page 

In [28]:
df = pd.DataFrame(all_jobs)

df.shape
df=df.drop_duplicates()
df.shape

df.isnull().sum()

df=df.dropna(subset=["Title","Company","Location"])
df = df.reset_index(drop=True)

In [29]:
df.isnull().sum()

Role           0
Title          0
Company        0
Location       0
Salary         0
Description    0
dtype: int64

In [30]:
df=df.dropna(subset=["Title","Company","Location"])
df = df.reset_index(drop=True)

In [31]:
for salary in df["Salary"].head(5):
    print(repr(salary))  # Show hidden characters

'$123823.45 - $123823.45'
'$139447.53 - $139447.53'
'$126307.09 - $126307.09'
'$139645.03 - $139645.03'
'$64259.04 - $64259.04'


In [32]:
import re

def extract_salary_range(salary):
    if isinstance(salary, str) and '-' in salary:
        try:
            # Remove $ and commas, then split
            salary = salary.replace('$', '').replace(',', '')
            min_salary, max_salary = salary.split('-')
            return float(min_salary.strip()), float(max_salary.strip())
        except:
            return None, None
    return None, None

# Apply the function
df["Salary_Min"], df["Salary_Max"] = zip(*df["Salary"].map(extract_salary_range))

In [33]:
df=df.reset_index(drop=True)
df.head()

,Role,Title,Company,Location,Salary,Description,Salary_Min,Salary_Max
0,Software Engineer,Signals Software Engineer,Lockheed Martin,"Harmans, Anne Arundel County",$123823.45 - $123823.45,"Job ID: 681944BR Date posted: Mar. 21, 2025 De...",123823.45,123823.45
1,Software Engineer,Cyber Software Engineer,Lockheed Martin,"Fort George G Meade, Anne Arundel County",$139447.53 - $139447.53,"Job ID: 682998BR Date posted: Dec. 24, 2024 De...",139447.53,139447.53
2,Software Engineer,Cyber Software Engineer Staff,Lockheed Martin,"Savage, Anne Arundel County",$126307.09 - $126307.09,"Job ID: 685238BR Date posted: Mar. 20, 2025 De...",126307.09,126307.09
3,Software Engineer,Cyber Software Engineer Sr,Lockheed Martin,"Savage, Anne Arundel County",$139645.03 - $139645.03,"Job ID: 685235BR Date posted: Mar. 22, 2025 De...",139645.03,139645.03
4,Software Engineer,Scene Generation/Software Engineer,BlueHalo LLC,"Redstone Arsenal, Madison County",$64259.04 - $64259.04,"BlueHalo is seeking a Jr., Mid. or Sr. level E...",64259.04,64259.04


In [34]:
df["Average_Salary"]=df[["Salary_Min","Salary_Max"]].mean(axis=1)
df.head()

,Role,Title,Company,Location,Salary,Description,Salary_Min,Salary_Max,Average_Salary
0,Software Engineer,Signals Software Engineer,Lockheed Martin,"Harmans, Anne Arundel County",$123823.45 - $123823.45,"Job ID: 681944BR Date posted: Mar. 21, 2025 De...",123823.45,123823.45,123823.45
1,Software Engineer,Cyber Software Engineer,Lockheed Martin,"Fort George G Meade, Anne Arundel County",$139447.53 - $139447.53,"Job ID: 682998BR Date posted: Dec. 24, 2024 De...",139447.53,139447.53,139447.53
2,Software Engineer,Cyber Software Engineer Staff,Lockheed Martin,"Savage, Anne Arundel County",$126307.09 - $126307.09,"Job ID: 685238BR Date posted: Mar. 20, 2025 De...",126307.09,126307.09,126307.09
3,Software Engineer,Cyber Software Engineer Sr,Lockheed Martin,"Savage, Anne Arundel County",$139645.03 - $139645.03,"Job ID: 685235BR Date posted: Mar. 22, 2025 De...",139645.03,139645.03,139645.03
4,Software Engineer,Scene Generation/Software Engineer,BlueHalo LLC,"Redstone Arsenal, Madison County",$64259.04 - $64259.04,"BlueHalo is seeking a Jr., Mid. or Sr. level E...",64259.04,64259.04,64259.04


In [35]:
df.head()

,Role,Title,Company,Location,Salary,Description,Salary_Min,Salary_Max,Average_Salary
0,Software Engineer,Signals Software Engineer,Lockheed Martin,"Harmans, Anne Arundel County",$123823.45 - $123823.45,"Job ID: 681944BR Date posted: Mar. 21, 2025 De...",123823.45,123823.45,123823.45
1,Software Engineer,Cyber Software Engineer,Lockheed Martin,"Fort George G Meade, Anne Arundel County",$139447.53 - $139447.53,"Job ID: 682998BR Date posted: Dec. 24, 2024 De...",139447.53,139447.53,139447.53
2,Software Engineer,Cyber Software Engineer Staff,Lockheed Martin,"Savage, Anne Arundel County",$126307.09 - $126307.09,"Job ID: 685238BR Date posted: Mar. 20, 2025 De...",126307.09,126307.09,126307.09
3,Software Engineer,Cyber Software Engineer Sr,Lockheed Martin,"Savage, Anne Arundel County",$139645.03 - $139645.03,"Job ID: 685235BR Date posted: Mar. 22, 2025 De...",139645.03,139645.03,139645.03
4,Software Engineer,Scene Generation/Software Engineer,BlueHalo LLC,"Redstone Arsenal, Madison County",$64259.04 - $64259.04,"BlueHalo is seeking a Jr., Mid. or Sr. level E...",64259.04,64259.04,64259.04


In [36]:
# ✅ Save to CSV
df.to_csv("adzuna_job_listings.csv", index=False)

print("\n✅ Data saved to adzuna_job_listings.csv")
print("📊 Total jobs collected:", len(df))

# Preview
df.head()


✅ Data saved to adzuna_job_listings.csv
📊 Total jobs collected: 10400


,Role,Title,Company,Location,Salary,Description,Salary_Min,Salary_Max,Average_Salary
0,Software Engineer,Signals Software Engineer,Lockheed Martin,"Harmans, Anne Arundel County",$123823.45 - $123823.45,"Job ID: 681944BR Date posted: Mar. 21, 2025 De...",123823.45,123823.45,123823.45
1,Software Engineer,Cyber Software Engineer,Lockheed Martin,"Fort George G Meade, Anne Arundel County",$139447.53 - $139447.53,"Job ID: 682998BR Date posted: Dec. 24, 2024 De...",139447.53,139447.53,139447.53
2,Software Engineer,Cyber Software Engineer Staff,Lockheed Martin,"Savage, Anne Arundel County",$126307.09 - $126307.09,"Job ID: 685238BR Date posted: Mar. 20, 2025 De...",126307.09,126307.09,126307.09
3,Software Engineer,Cyber Software Engineer Sr,Lockheed Martin,"Savage, Anne Arundel County",$139645.03 - $139645.03,"Job ID: 685235BR Date posted: Mar. 22, 2025 De...",139645.03,139645.03,139645.03
4,Software Engineer,Scene Generation/Software Engineer,BlueHalo LLC,"Redstone Arsenal, Madison County",$64259.04 - $64259.04,"BlueHalo is seeking a Jr., Mid. or Sr. level E...",64259.04,64259.04,64259.04
